# Conditional Random Fields for Surface Segmentation Feasibility Demo 18/08/2020
## Tumi Moeng

## Data Cleaning
First, the data we received needs to be cleaned and the surface segmentation
 forms need to be generated to be used by the models. To this end I created a
 class called 'DataCleaner' which takes the data we received and filters out
 that which we need from that which we dont need and generates some more data
 that we need.<br>
 This class takes the data in the following form:<br>
 ngezinkonzo&emsp;khonzo&emsp;P&emsp;[RelConc]-nga[NPre]-i[NPrePre]-zin[BPre]-konzo[NStem]<br>
 And converts it to the following form:<br>
 ngezinkonzo | nge-zin-konzo | nga[NPre]i[NPrePre]zin[BPre]konzo[NStem]

In [1]:
from morphology.DataCleaner import DataCleaner
languages = ["zulu", "swati", "ndebele", "xhosa"]

for lang in languages:
    print("Language: " + lang)
    inputFile = DataCleaner("morphology\\"+lang + ".train.conll")
    inputFile.reformat("morphology\\"+lang + ".clean.train")
    inputFile = DataCleaner("morphology\\"+lang + ".test.conll")
    inputFile.reformat("morphology\\"+lang + ".clean.test")
    print(lang + " cleaning complete.\n#############################################")

Language: zulu


FileNotFoundError: [Errno 2] No such file or directory: 'zulu.train.conll'

## Preparation for Hyper-parameter Optimisation
Secondly, in preparation for the hyper-parameter optimisation that we will need to do
 to ensure our models are as good as possible we needed to be able to develop a test /
 validation set to be used in the optimisation process. To this end, I created a class
 called 'ValidationSetCreation' that extracts about 10% of the data from the training set
 and puts it into a validation set. This 10% represents roughly the same amount of entries
 as the test set contains.

In [9]:
from morphology.ValidationSetCreation import ValidationSet

for lang in languages:
    print("Language: " + lang)
    file_name ="morphology\\" + lang + ".clean.train.conll"
    # print(file_name)
    inputFile = ValidationSet(file_name)
    file_name = "morphology\\"+lang + ".clean.dev.conll"
    # print(file_name)
    inputFile.create_validation_set(file_name)
    print(lang + " validation set complete.\n#############################################")


Language: zulu
3233
zulu validation set complete.
#############################################
Language: swati
5159
swati validation set complete.
#############################################
Language: ndebele
2985
ndebele validation set complete.
#############################################
Language: xhosa
3262
xhosa validation set complete.
#############################################


## Conditional Random Field using SKLearn_CRFSuite
Finally, we are now able to train the CRF using the data we have developed and
train the CRF on this data. We give it input as the string of the language it
is to be trained on and it will give 2 outputs. The first is a list of lists of
what the CRF predicted the labels will be for the words in the test set. The second
 is a list of lists of what the actual labels for the word in the test set are. Both
 inner lists of labels occur in the following form:<br>
 For the word 'komthombo' which segments to 'ko-m-thombo the inner list would be
 [B,E,S,B,M,M,M,M,E] where:<br>
 B = Beginning of Segment<br>
 M = Middle of Segment<br>
 E = End of Segment<br>
 S = Single Length Segment<br>
 The results method which takes in the list of predicted answers and actual answers
 can be used to print to console the results of the CRF such as the <b>precision</b>,
 <b>recall</b> and <b>F1-Score</b>.

In [5]:
import sklearn_crfsuite
from sklearn.metrics import accuracy_score

from BaselineCRF import BaselineCRF
crf = BaselineCRF("zulu")
predicted_ans, actual_ans = crf.surface_segmentation()
print("First Value of Predicted Answer List: "+str(predicted_ans[0]))
print("First Value of Actual Answer List: "+str(actual_ans[0]))
crf.surface_segmentation_results(predicted_ans, actual_ans)
print(accuracy_score(predicted_ans, actual_ans))

Data Collected in -1.0572801000089385
Beginning Feature Computation and Model Optimisation
Features Successfully Computed & Model Optimised -0.0006557000451721251
X_Training: [{'uni_u': 1, 'bi_uy': 1, 'vowel': 1, 'lower': 1}, {'uni_y': 1, 'bi_uy': 1, 'bi_yi': 1, 'tri_uyi': 1, 'four_uyik': 1, 'const': 1, 'lower': 1}, {'uni_i': 1, 'bi_yi': 1, 'bi_ik': 1, 'tri_yik': 1, 'four_yikh': 1, 'four_uyik': 1, 'five_uyikh': 1, 'six_uyikho': 1, 'vowel': 1, 'lower': 1}, {'uni_k': 1, 'bi_ik': 1, 'bi_kh': 1, 'tri_ikh': 1, 'four_ikho': 1, 'four_yikh': 1, 'five_yikho': 1, 'six_uyikho': 1, 'six_yikhok': 1, 'const': 1, 'lower': 1}, {'uni_h': 1, 'bi_kh': 1, 'bi_ho': 1, 'tri_kho': 1, 'four_khok': 1, 'four_ikho': 1, 'five_ikhok': 1, 'six_yikhok': 1, 'six_ikhokh': 1, 'const': 1, 'lower': 1}, {'uni_o': 1, 'bi_ho': 1, 'bi_ok': 1, 'tri_hok': 1, 'four_hokh': 1, 'four_khok': 1, 'five_khokh': 1, 'six_ikhokh': 1, 'six_khokhe': 1, 'vowel': 1, 'lower': 1}, {'uni_k': 1, 'bi_ok': 1, 'bi_kh': 1, 'tri_okh': 1, 'four_okhe':

ValueError: You appear to be using a legacy multi-label data representation. Sequence of sequences are no longer supported; use a binary array or sparse matrix instead - the MultiLabelBinarizer transformer can convert to this format.

In [11]:
import sklearn
sklearn.__file__
from sklearn.metrics import accuracy_score
print(accuracy_score([0,1,0,1,1,1,1],[1,1,0,1,0,1,1]))



0.7142857142857143
